In [172]:
import pandas as pd
from gluonts.dataset.pandas import PandasDataset

df = pd.read_csv('Thesis Tracking Data 20230919.csv')
df = df[['Timestamp', 'Latitude', 'Longitude']]
# df = df.rename(columns={'Timestamp': 'timestamp'})
# df['timestamp'] = pd.to_datetime(df['timestamp'])

df = df.astype({'Timestamp': 'string'})
df['Timestamp'] = df['Timestamp'].str.replace(' \+0000', '')
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# df.set_index('timestamp')

gap_fill_df = pd.DataFrame({
    'Timestamp': pd.date_range(df['Timestamp'].min(), df['Timestamp'].max(), freq='1s')
})

df = pd.merge(df, gap_fill_df, on='Timestamp', how='outer')
df = df.sort_values(by=['Timestamp'])
df['Latitude'] = df['Latitude'].interpolate()
df['Longitude'] = df['Longitude'].interpolate()

df['Item_ID'] = '2'
df.loc[0:2200, 'Item_ID'] = '1'

# lat_ds = PandasDataset.from_long_dataframe(df, target='Latitude', item_id='Item_ID', timestamp='Timestamp', freq='S')
#long_ds = PandasDataset.from_long_dataframe(df, target='Longitude', item_id='Item_ID', timestamp='Timestamp', freq='S')

# lat_ds

lat_df = df[['Timestamp', 'Latitude', 'Item_ID']]
long_df = df[['Timestamp', 'Longitude', 'Item_ID']]

/var/folders/4b/fm5gnnzx6qqc7t0l504grbjm0000gn/T/ipykernel_49346/3743159901.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Timestamp'] = df['Timestamp'].str.replace(' \+0000', '')


In [110]:
from gluonts.itertools import Map

class ProcessStartField():
    ts_id = 0
    
    def __call__(self, data):
        data["start"] = data["start"].to_timestamp()
        data["feat_static_cat"] = [self.ts_id]
        self.ts_id += 1
        
        return data

process_start = ProcessStartField()

# lat_list_ds = list(Map(process_start, lat_ds))
# long_list_ds = list(Map(process_start, long_ds))

# lat_list_ds

AssertionError: Dataframe index is not uniformly spaced. If your dataframe contains data from multiple series in the same column ("long" format), consider constructing the dataset with `PandasDataset.from_long_dataframe` instead.

In [173]:
# Logic behind this: each row will be used as test data, whose validation data is 
# found in the succeeding row
# It already incorporates one loop around Line A as a starting point

lat_df['Target'] = [None if x < 2250 else df['Latitude'].iloc[2200:x].tolist() for x in range(len(df))]
long_df['Target'] = [None if x < 2250 else df['Longitude'].iloc[2200:x].tolist() for x in range(len(df))]

# To ensure that the prediction length is 10 gps coordinates long, we take a sample every 10 seconds
lat_df = lat_df.iloc[::10, :]
long_df = long_df.iloc[::10, :]

lat_df = lat_df[['Timestamp', 'Item_ID', 'Target']]
long_df = long_df[['Timestamp', 'Item_ID', 'Target']]

lat_df['feat_static_cat'] = [[0]] * len(lat_df)
long_df['feat_static_cat'] = [[0]] * len(long_df)

In [177]:
from datasets import Dataset, Features, Value, Sequence

features = Features(
    {
        'Timestamp': Value('timestamp[s]'),
        'Target': Sequence(Value('float64')),
        'feat_static_cat': Sequence(Value('uint64')),
        'Item_ID': Value('string')
    }
)

lat_df.dropna(axis=0, how='any', inplace=True)
long_df.dropna(axis=0, how='any', inplace=True)

lat_ds = Dataset.from_pandas(lat_df)
long_ds = Dataset.from_pandas(long_df)

lat_df_test = lat_df.copy()
long_df_test = long_df.copy()

lat_df_test['Target'] = lat_df_test['Target'].shift(-1)
long_df_test['Target'] = lat_df_test['Target'].shift(-1)

lat_df_test.dropna(axis=0, how='any', inplace=True)
long_df_test.dropna(axis=0, how='any', inplace=True)

lat_test_ds = Dataset.from_pandas(lat_df_test)
long_test_ds = Dataset.from_pandas(long_df_test)

lat_dataset = {'train': lat_ds, 'test': lat_test_ds}
long_dataset = {'train': long_ds, 'test': long_test_ds}

In [127]:
!python3 -m pip install transformers datasets evaluate accelerate gluonts ujson

In [178]:
# Update the start period to a pandas period
from functools import partial

def convert_to_pandas_period(date, freq):
    return pd.Period(date, freq)

def transform_start_field(batch, freq):
    batch['start'] = [convert_to_pandas_period(date, freq) for date in batch["start"]]

freq='10s'

lat_dataset['train'].set_transform(partial(transform_start_field, freq=freq))
lat_dataset['test'].set_transform(partial(transform_start_field, freq=freq))

long_dataset['train'].set_transform(partial(transform_start_field, freq=freq))
long_dataset['test'].set_transform(partial(transform_start_field, freq=freq))


In [183]:
from gluonts.time_feature import get_lags_for_frequency, time_features_from_frequency_str

lags_seq = get_lags_for_frequency(freq)
time_features = time_features_from_frequency_str(freq)

print(lat_dataset['train'])



Dataset({
    features: ['Timestamp', 'Item_ID', 'Target', 'feat_static_cat', '__index_level_0__'],
    num_rows: 24
})


In [185]:
# Defining the transformer model
prediction_length = 10

from transformers import TimeSeriesTransformerConfig, TimeSeriesTransformerForPrediction

lat_config = TimeSeriesTransformerConfig(
    prediction_length=prediction_length,
    context_length=2000,
    lags_sequence=lags_seq,
    num_time_features=len(time_features),
    num_static_categorical_features=1,
    cardinality=[len(lat_dataset['train'])],
    embedding_dimension=[1],

    encoder_layers=4,
    decoder_layers=4,
    d_model=32
)

long_config = TimeSeriesTransformerConfig(
    prediction_length=prediction_length,
    context_length=2000,
    lags_sequence=lags_seq,
    num_time_features=len(time_features),
    num_static_categorical_features=1,
    cardinality=[len(long_dataset['train'])],
    embedding_dimension=[1],

    encoder_layers=4,
    decoder_layers=4,
    d_model=32
)

lat_model = TimeSeriesTransformerForPrediction(lat_config)
long_model = TimeSeriesTransformerForPrediction(long_config)

In [186]:
from gluonts.time_feature import (
    time_features_from_frequency_str,
    TimeFeature,
    get_lags_for_frequency,
)
from gluonts.dataset.field_names import FieldName
from gluonts.transform import (
    AddAgeFeature,
    AddObservedValuesIndicator,
    AddTimeFeatures,
    AsNumpyArray,
    Chain,
    ExpectedNumInstanceSampler,
    InstanceSplitter,
    RemoveFields,
    SelectFields,
    SetField,
    TestSplitSampler,
    Transformation,
    ValidationSplitSampler,
    VstackFeatures,
    RenameFields,
)

In [187]:
from transformers import PretrainedConfig


def create_transformation(freq: str, config: PretrainedConfig) -> Transformation:
    remove_field_names = []
    if config.num_static_real_features == 0:
        remove_field_names.append(FieldName.FEAT_STATIC_REAL)
    if config.num_dynamic_real_features == 0:
        remove_field_names.append(FieldName.FEAT_DYNAMIC_REAL)
    if config.num_static_categorical_features == 0:
        remove_field_names.append(FieldName.FEAT_STATIC_CAT)

    # a bit like torchvision.transforms.Compose
    return Chain(
        # step 1: remove static/dynamic fields if not specified
        [RemoveFields(field_names=remove_field_names)]
        # step 2: convert the data to NumPy (potentially not needed)
        + (
            [
                AsNumpyArray(
                    field=FieldName.FEAT_STATIC_CAT,
                    expected_ndim=1,
                    dtype=int,
                )
            ]
            if config.num_static_categorical_features > 0
            else []
        )
        + (
            [
                AsNumpyArray(
                    field=FieldName.FEAT_STATIC_REAL,
                    expected_ndim=1,
                )
            ]
            if config.num_static_real_features > 0
            else []
        )
        + [
            AsNumpyArray(
                field=FieldName.TARGET,
                # we expect an extra dim for the multivariate case:
                expected_ndim=1, ## cs: from expected_ndim=1 if config.input_size == 1 else 2,
            ),
            # step 3: handle the NaN's by filling in the target with zero
            # and return the mask (which is in the observed values)
            # true for observed values, false for nan's
            # the decoder uses this mask (no loss is incurred for unobserved values)
            # see loss_weights inside the xxxForPrediction model
            AddObservedValuesIndicator(
                target_field=FieldName.TARGET,
                output_field=FieldName.OBSERVED_VALUES,
            ),
            # step 4: add temporal features based on freq of the dataset
            # month of year in the case when freq="M"
            # these serve as positional encodings
            AddTimeFeatures(
                start_field=FieldName.START,
                target_field=FieldName.TARGET,
                output_field=FieldName.FEAT_TIME,
                time_features=time_features_from_frequency_str(freq),
                pred_length=config.prediction_length,
            ),
#             # step 5: add another temporal feature (just a single number)
#             # tells the model where in the life the value of the time series is
#             # sort of running counter
#             AddAgeFeature(
#                 target_field=FieldName.TARGET,
#                 output_field=FieldName.FEAT_AGE,
#                 pred_length=config.prediction_length,
#                 log_scale=True,
#             ),
            # step 6: vertically stack all the temporal features into the key FEAT_TIME
            VstackFeatures(
                output_field=FieldName.FEAT_TIME,
                input_fields=[FieldName.FEAT_TIME]  ## cs: from "[FieldName.FEAT_TIME, FieldName.FEAT_AGE]"
                + (
                    [FieldName.FEAT_DYNAMIC_REAL]
                    if config.num_dynamic_real_features > 0
                    else []
                ),
            ),
            # step 7: rename to match HuggingFace names
            RenameFields(
                mapping={
                    FieldName.FEAT_STATIC_CAT: "static_categorical_features",
                    FieldName.FEAT_STATIC_REAL: "static_real_features",
                    FieldName.FEAT_TIME: "time_features",
                    FieldName.TARGET: "values",
                    FieldName.OBSERVED_VALUES: "observed_mask",
                }
            ),
        ]
    )

# print(len(FieldName.FEAT_TIME),len(FieldName.FEAT_TIME), len(FieldName.FEAT_DYNAMIC_REAL))
for i in [FieldName.FEAT_STATIC_CAT, FieldName.FEAT_STATIC_REAL, FieldName.FEAT_TIME, FieldName.TARGET, FieldName.OBSERVED_VALUES]:
    print(i, len(i))

feat_static_cat 15
feat_static_real 16
time_feat 9
target 6
observed_values 15


# INSTANCE SPLITTER

In [188]:
from gluonts.transform.sampler import InstanceSampler
from typing import Optional


def create_instance_splitter(
    config: PretrainedConfig,
    mode: str,
    train_sampler: Optional[InstanceSampler] = None,
    validation_sampler: Optional[InstanceSampler] = None,
) -> Transformation:
    assert mode in ["train", "validation", "test"]

    instance_sampler = {
        "train": train_sampler
        or ExpectedNumInstanceSampler(
            num_instances=1.0, min_future=config.prediction_length
        ),
        "validation": validation_sampler
        or ValidationSplitSampler(min_future=config.prediction_length),
        "test": TestSplitSampler(),
    }[mode]

    return InstanceSplitter(
        target_field="values",
        is_pad_field=FieldName.IS_PAD,
        start_field=FieldName.START,
        forecast_start_field=FieldName.FORECAST_START,
        instance_sampler=instance_sampler,
        past_length=config.context_length + max(config.lags_sequence),
        future_length=config.prediction_length,
        time_series_fields=["time_features", "observed_mask"],
    )

In [125]:
!python3 -m pip install pytorch_lightning

  Obtaining dependency information for pytorch_lightning from https://files.pythonhosted.org/packages/fa/c7/18aca7e74b6c4bb99ceb76a7742716543f040834b8440acad4afaf528e46/pytorch_lightning-2.0.9-py3-none-any.whl.metadata
  Obtaining dependency information for torchmetrics>=0.7.0 from https://files.pythonhosted.org/packages/a3/88/cc27059747ddecff744826e38014822023cbfff4ca079a6ee9a96602dd0b/torchmetrics-1.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for lightning-utilities>=0.7.0 from https://files.pythonhosted.org/packages/46/ee/8641eeb6a062f383b7d6875604e1f3f83bd2c93a0b4dbcabd3150b32de6e/lightning_utilities-0.9.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.7/727.7 kB 10.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 17.4 MB/s eta 0:00:00a 0:00:01


# Creating Data Loader

In [189]:
from gluonts.itertools import Cyclic, IterableSlice, PseudoShuffled
from gluonts.torch.util import IterableDataset
from torch.utils.data import DataLoader

from typing import Iterable


def create_train_dataloader(
    config: PretrainedConfig,
    freq,
    data,
    batch_size: int,
    num_batches_per_epoch: int,
    shuffle_buffer_length: Optional[int] = None,
    **kwargs,
) -> Iterable:
    PREDICTION_INPUT_NAMES = [
        "past_time_features",
        "past_values",
        "past_observed_mask",
        "future_time_features",
    ]
    if config.num_static_categorical_features > 0:
        PREDICTION_INPUT_NAMES.append("static_categorical_features")

    if config.num_static_real_features > 0:
        PREDICTION_INPUT_NAMES.append("static_real_features")

    TRAINING_INPUT_NAMES = PREDICTION_INPUT_NAMES + [
        "future_values",
        "future_observed_mask",
    ]

    transformation = create_transformation(freq, config)
    transformed_data = transformation.apply(data, is_train=True)

    # we initialize a Training instance
    instance_splitter = create_instance_splitter(config, "train") + SelectFields(
        TRAINING_INPUT_NAMES
    )

    # the instance splitter will sample a window of
    # context length + lags + prediction length (from the 366 possible transformed time series)
    # randomly from within the target time series and return an iterator.
    training_instances = instance_splitter.apply(
        Cyclic(transformed_data)
        if shuffle_buffer_length is None
        else PseudoShuffled(
            Cyclic(transformed_data),
            shuffle_buffer_length=shuffle_buffer_length,
        )
    )

    # from the training instances iterator we now return a Dataloader which will
    # continue to sample random windows for as long as it is called
    # to return batch_size of the appropriate tensors ready for training!
    return IterableSlice(
        iter(
            DataLoader(
                IterableDataset(training_instances),
                batch_size=batch_size,
                **kwargs,
            )
        ),
        num_batches_per_epoch,
    )

def create_test_dataloader(
    config: PretrainedConfig,
    freq,
    data,
    batch_size: int,
    **kwargs,
):
    PREDICTION_INPUT_NAMES = [
        "past_time_features",
        "past_values",
        "past_observed_mask",
        "future_time_features",
    ]
    if config.num_static_categorical_features > 0:
        PREDICTION_INPUT_NAMES.append("static_categorical_features")

    if config.num_static_real_features > 0:
        PREDICTION_INPUT_NAMES.append("static_real_features")

    transformation = create_transformation(freq, config)
    transformed_data = transformation.apply(data, is_train=False)

    # we create a Test Instance splitter which will sample the very last
    # context window seen during training only for the encoder.
    instance_sampler = create_instance_splitter(config, "test") + SelectFields(
        PREDICTION_INPUT_NAMES
    )

    # we apply the transformations in test mode
    testing_instances = instance_sampler.apply(transformed_data, is_train=False)

    # This returns a Dataloader which will go over the dataset once.
    return DataLoader(
        IterableDataset(testing_instances), batch_size=batch_size, **kwargs
    )

ImportError: cannot import name 'IterableDataset' from 'gluonts.torch.util' (/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gluonts/torch/util.py)

In [ ]:
# Instantiating Dataloaders
batch_size = 256

lat_train_dataloader = create_train_dataloader(
    config=lat_config,
    freq=freq,
    data=lat_dataset['train'],
    batch_size=batch_size,
    num_batches_per_epoch=100,
)

long_train_dataloader = create_train_dataloader(
    config=long_config,
    freq=freq,
    data=long_dataset['train'],
    batch_size=batch_size,
    num_batches_per_epoch=100,
)

lat_test_dataloader = create_test_dataloader(
    config=lat_config,
    freq=freq,
    data=lat_dataset['test'],
    batch_size=64,
)

long_test_dataloader = create_test_dataloader(
    config=long_config,
    freq=freq,
    data=long_dataset['test'],
    batch_size=64,
)

# Forward Pass

In [ ]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") ## cs
print(device)

lat_batch = next(iter(lat_train_dataloader))
long_batch = next(iter(long_train_dataloader))

# perform forward pass
lat_outputs = lat_model(
    past_values=lat_batch["past_values"],
    past_time_features=lat_batch["past_time_features"],
    past_observed_mask=lat_batch["past_observed_mask"],
    static_categorical_features=lat_batch["static_categorical_features"]
    if lat_config.num_static_categorical_features > 0
    else None,
    static_real_features=lat_batch["static_real_features"]
    if lat_config.num_static_real_features > 0
    else torch.zeros((batch_size, 0), dtype=torch.float32, device=device),  ## cs: changed from "else None"
    future_values=lat_batch["future_values"],
    future_time_features=lat_batch["future_time_features"],
    future_observed_mask=lat_batch["future_observed_mask"],
    output_hidden_states=True,
)

long_outputs = long_model(
    past_values=long_batch["past_values"],
    past_time_features=long_batch["past_time_features"],
    past_observed_mask=long_batch["past_observed_mask"],
    static_categorical_features=long_batch["static_categorical_features"]
    if long_config.num_static_categorical_features > 0
    else None,
    static_real_features=long_batch["static_real_features"]
    if long_config.num_static_real_features > 0
    else torch.zeros((batch_size, 0), dtype=torch.float32, device=device),  ## cs: changed from "else None"
    future_values=long_batch["future_values"],
    future_time_features=long_batch["future_time_features"],
    future_observed_mask=long_batch["future_observed_mask"],
    output_hidden_states=True,
)

# Model Training

### Latitude Model Training

In [ ]:
from accelerate import Accelerator
from torch.optim import AdamW

lat_accelerator = Accelerator()
lat_device = lat_accelerator.device
lat_device = "cpu"  ## cs

lat_model.to(lat_device)
long_optimizer = AdamW(lat_model.parameters(), lr=6e-4, betas=(0.9, 0.95), weight_decay=1e-1)

# model, optimizer, train_dataloader = accelerator.prepare(
#     model,
#     optimizer,
#     train_dataloader,
# )

lat_model.train()
for epoch in range(40):
    for idx, batch in enumerate(lat_train_dataloader):
        long_optimizer.zero_grad()
        outputs = lat_model(
            static_categorical_features=batch["static_categorical_features"].to(lat_device)
            if lat_config.num_static_categorical_features > 0
            else None,
            static_real_features=batch["static_real_features"].to(lat_device)
            if lat_config.num_static_real_features > 0
            else torch.zeros((batch_size, 0), dtype=torch.float32, device=lat_device).to(lat_device),  ## cs: changed from "else None"
            past_time_features=batch["past_time_features"].to(lat_device),
            past_values=batch["past_values"].to(lat_device),
            future_time_features=batch["future_time_features"].to(lat_device),
            future_values=batch["future_values"].to(lat_device),
            past_observed_mask=batch["past_observed_mask"].to(lat_device),
            future_observed_mask=batch["future_observed_mask"].to(lat_device),
        )
        
        loss = outputs.loss

        # Backpropagation
        lat_accelerator.backward(loss)
        long_optimizer.step()

        if idx % 100 == 0:
            print(loss.item())

### Longitude Model Training

In [ ]:
from accelerate import Accelerator
from torch.optim import AdamW

long_accelerator = Accelerator()
long_device = long_accelerator.device
long_device = "cpu"  ## cs

long_model.to(device)
long_optimizer = AdamW(long_model.parameters(), lr=6e-4, betas=(0.9, 0.95), weight_decay=1e-1)

# model, optimizer, train_dataloader = accelerator.prepare(
#     model,
#     optimizer,
#     train_dataloader,
# )

long_model.train()
for epoch in range(40):
    for idx, batch in enumerate(long_train_dataloader):
        long_optimizer.zero_grad()
        outputs = long_model(
            static_categorical_features=batch["static_categorical_features"].to(long_device)
            if long_config.num_static_categorical_features > 0
            else None,
            static_real_features=batch["static_real_features"].to(long_device)
            if long_config.num_static_real_features > 0
            else torch.zeros((batch_size, 0), dtype=torch.float32, device=long_device).to(long_device),  ## cs: changed from "else None"
            past_time_features=batch["past_time_features"].to(long_device),
            past_values=batch["past_values"].to(long_device),
            future_time_features=batch["future_time_features"].to(long_device),
            future_values=batch["future_values"].to(long_device),
            past_observed_mask=batch["past_observed_mask"].to(long_device),
            future_observed_mask=batch["future_observed_mask"].to(long_device),
        )
        
        loss = outputs.loss

        # Backpropagation
        long_accelerator.backward(loss)
        long_optimizer.step()

        if idx % 100 == 0:
            print(loss.item())

# Inference

### Latitude Inference

In [ ]:
lat_model.eval()

lat_forecasts = []

for batch in lat_test_dataloader:
    outputs = lat_model.generate(
        static_categorical_features=batch["static_categorical_features"].to(lat_device)
        if lat_config.num_static_categorical_features > 0
        else None,
        static_real_features=batch["static_real_features"].to(lat_device)
        if lat_config.num_static_real_features > 0
        else torch.zeros((3, 0), dtype=torch.float32, device=lat_device).to(lat_device),  ## cs: changed from "else None"
        past_time_features=batch["past_time_features"].to(lat_device),
        past_values=batch["past_values"].to(lat_device),
        future_time_features=batch["future_time_features"].to(lat_device),
        past_observed_mask=batch["past_observed_mask"].to(lat_device),
    )
    lat_forecasts.append(outputs.sequences.cpu().numpy())

In [147]:
import numpy as np

lat_forecasts = np.vstack(lat_forecasts)

In [ ]:
from evaluate import load
from gluonts.time_feature import get_seasonality

import numpy as np

lat_mase_metric = load("evaluate-metric/mase")
lat_smape_metric = load("evaluate-metric/smape")
lat_mse_metric = load("evaluate-metric/mse")

lat_forecast_median = np.median(lat_forecasts, 1)

lat_mase_metrics = []
lat_smape_metrics = []
lat_mse_metrics = []
for item_id, ts in enumerate(lat_dataset['test']):
    training_data = ts["target"][:-prediction_length]
    ground_truth = ts["target"][-prediction_length:]
#     mase = mase_metric.compute(   cs: commented out cuz not working
#         predictions=forecast_median[item_id],
#         references=np.array(ground_truth),
#         training=np.array(training_data),
#         periodicity=get_seasonality(freq),
#     )
          
#     mase_metrics.append(mase["mase"])
    print(lat_forecast_median[item_id])
    print(np.array(ground_truth))

    long_smape = lat_smape_metric.compute(
        predictions=lat_forecast_median[item_id],
        references=np.array(ground_truth),
    )
    lat_smape_metrics.append(long_smape["smape"])
    
    lat_mse = lat_mse_metric.compute(predictions=lat_forecast_median[item_id], references=np.array(ground_truth))  ## cs: added for mse
    lat_mse_metrics.append(lat_mse["mse"])

print(f"sMAPE: {np.mean(lat_smape_metrics)}, MSE: {np.mean(lat_mse_metrics)}")

### Longitude Inference

In [ ]:
long_model.eval()

long_forecasts = []

for batch in long_test_dataloader:
    outputs = long_model.generate(
        static_categorical_features=batch["static_categorical_features"].to(long_device)
        if long_config.num_static_categorical_features > 0
        else None,
        static_real_features=batch["static_real_features"].to(long_device)
        if long_config.num_static_real_features > 0
        else torch.zeros((3, 0), dtype=torch.float32, device=long_device).to(long_device),  ## cs: changed from "else None"
        past_time_features=batch["past_time_features"].to(long_device),
        past_values=batch["past_values"].to(long_device),
        future_time_features=batch["future_time_features"].to(long_device),
        past_observed_mask=batch["past_observed_mask"].to(long_device),
    )
    long_forecasts.append(outputs.sequences.cpu().numpy())

In [ ]:
import numpy as np

long_forecasts = np.vstack(long_forecasts)

In [ ]:
from evaluate import load
from gluonts.time_feature import get_seasonality

import numpy as np

long_mase_metric = load("evaluate-metric/mase")
long_smape_metric = load("evaluate-metric/smape")
long_mse_metric = load("evaluate-metric/mse")

long_forecast_median = np.median(lat_forecasts, 1)

long_mase_metrics = []
long_smape_metrics = []
long_mse_metrics = []
for item_id, ts in enumerate(long_dataset['test']):
    training_data = ts["target"][:-prediction_length]
    ground_truth = ts["target"][-prediction_length:]
#     mase = mase_metric.compute(   cs: commented out cuz not working
#         predictions=forecast_median[item_id],
#         references=np.array(ground_truth),
#         training=np.array(training_data),
#         periodicity=get_seasonality(freq),
#     )
          
#     mase_metrics.append(mase["mase"])
    print(long_forecast_median[item_id])
    print(np.array(ground_truth))

    long_smape = lat_smape_metric.compute(
        predictions=lat_forecast_median[item_id],
        references=np.array(ground_truth),
    )
    lat_smape_metrics.append(long_smape["smape"])
    
    long_mse = long_mse_metric.compute(predictions=long_forecast_median[item_id], references=np.array(ground_truth))  ## cs: added for mse
    long_mse_metrics.append(long_mse["mse"])

print(f"sMAPE: {np.mean(long_smape_metrics)}, MSE: {np.mean(long_mse_metrics)}")

# Plotting Prediction

In [ ]:
import matplotlib.dates as mdates


def plot(ts_index):
    fig, ax = plt.subplots()

    index = pd.period_range(
        start=test_dataset[ts_index][FieldName.START],
        periods=len(test_dataset[ts_index][FieldName.TARGET]),
        freq=freq,
    ).to_timestamp()

    # Major ticks every half year, minor ticks every month,
#     ax.xaxis.set_major_locator(mdates.MonthLocator(bymonth=(1, 7)))  ## cs: removed month axis ticks to default as auto
#     ax.xaxis.set_minor_locator(mdates.MonthLocator())



#     ax.plot(  ## cs: replaced "-2*prediction_length" by "0" to show full time graph
#         index[-2 * prediction_length :],
#         test_dataset[ts_index]["target"][-2 * prediction_length :],
#         label="actual",
#     )
    ax.plot(
        index[0 :],
        test_dataset[ts_index]["target"][0 :],
        label="actual",
    )

    plt.plot(
        index[-prediction_length:],
        np.median(forecasts[ts_index], axis=0),
        label="median",
    )

    plt.fill_between(
        index[-prediction_length:],
        forecasts[ts_index].mean(0) - forecasts[ts_index].std(axis=0),
        forecasts[ts_index].mean(0) + forecasts[ts_index].std(axis=0),
        alpha=0.3,
        interpolate=True,
        label="+/- 1-std",
    )
    plt.legend()
    plt.show()

for i in range(3):
    plot(i)